# PROYECTO 1

IMPORTAMOS LIBRERIAS

In [2]:
import pandas as pd
import numpy as np
import json

# CARGA DE DATOS

In [3]:
# Funcion que carga un archivo JSON con líneas que contienen objetos JSON o diccionarios

def cargar_json(archivo):

# Inicializamos una lista vacía para almacenar los datos.
    lista = []                  

# Abrimos el archivo 'archivo' en modo de lectura ('r') con codificación UTF-8.
    with open(archivo, 'r', encoding='utf-8') as file:
        for num_linea, linea in enumerate(file, start=1):
# Iteramos a través de cada línea del archivo, manteniendo un contador 'num_linea'.
           
            try:
                data = json.loads(linea)          # Intentamos cargar la línea actual como un objeto JSON.
                lista.append(data)                # Agregamos el objeto JSON a la lista.
            except json.JSONDecodeError:          # Si la línea no es un objeto JSON válido y genera un error 'JSONDecodeError', continuamos con el siguiente paso.
         
                try:
                    data = eval(linea)              # Intentamos interpretar la línea como un diccionario utilizando 'eval'.
                    if isinstance(data, dict):      # Verificamos si 'data' es un diccionario.
                        lista.append(data)          # Agregamos el diccionario a la lista.
                except Exception as e:              # Si hay cualquier otro error durante el proceso, lo atrapamos y lo ignoramos.
                    pass

    df = pd.DataFrame(lista)        # Creamos un DataFrame a partir de la lista de datos obtenidos.
    return df

In [4]:
#Aca asignamos variables a nuestros archivos
archivo1 = 'Datasets/Datasets descomprimidos/australian_user_reviews.json.'
archivo2 = 'Datasets/Datasets descomprimidos/australian_users_items.json.'
archivo3 = 'Datasets/Datasets descomprimidos/output_steam_games.json.'

#Cargamos los archivos en un DataFrame
df_reviews = cargar_json(archivo1)
df_items = cargar_json(archivo2)
df_games = cargar_json(archivo3)

In [6]:
df_reviews.head(3)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."


In [7]:
df_items.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [8]:
df_games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
print(df_reviews.shape)
print(df_items.shape)
print(df_games.shape)

(25799, 3)
(88310, 5)
(120445, 13)


# DESANIDAMOS

In [10]:
df_exploded = df_reviews.explode('reviews')
df_normalized = pd.json_normalize(df_exploded['reviews'])

df_exploded.reset_index(drop=True, inplace=True)
df_normalized.reset_index(drop=True, inplace=True)

# Combinamos con el DataFrame original
df_reviews_limpio = pd.concat([df_exploded, df_normalized], axis=1).drop('reviews', axis=1)

#cambiamos por nan los espacios en blanco
df_reviews_limpio['funny'] = df_reviews_limpio['funny'].replace('', np.nan)
df_reviews_limpio['last_edited'] = df_reviews_limpio['last_edited'].replace('', np.nan)

#Cambiamos la columna review a tipo str
df_reviews_limpio['review'] = df_reviews_limpio['review'].astype(str)

In [11]:
df_explo = df_items.explode('items')
df_nor = pd.json_normalize(df_explo['items'])
df_explo.head()

df_explo.reset_index(drop=True, inplace=True)
df_nor.reset_index(drop=True, inplace=True)

# Combinamos con el DataFrame original
df_items_limpio = pd.concat([df_explo, df_nor], axis=1).drop('items', axis=1)

#los valores nulos los convertimos a cero
df_items_limpio['playtime_forever'] = df_items_limpio['playtime_forever'].fillna(0).astype(int)

#convertimos la columna playtime_forever a tipo entero
df_items_limpio['playtime_forever'] = df_items_limpio['playtime_forever'].astype(int)

# Eliminamos las filas que el playtime_forever es igual a cero
df_items_limpio = df_items_limpio[df_items_limpio['playtime_forever'] != 0]

# Borramos columnas que no nos sirvan

In [12]:
df_reviews_limpio = df_reviews_limpio.drop(columns= 'user_url')
df_items_limpio = df_items_limpio.drop(columns= ['user_url','steam_id','items_count','playtime_2weeks'])
df_games_limpio = df_games.drop(columns= ['reviews_url','title','url'])

# Le borramos los NaN

In [13]:
df_reviews_limpio = df_reviews_limpio.dropna(how='all', axis=0)
df_items_limpio = df_items_limpio.dropna(how='all', axis=0)
df_games_limpio = df_games_limpio.dropna(how='all', axis=0)

# Borramos los duplicados

In [14]:
df_games_limpio = df_games_limpio.drop_duplicates(subset=["id"])

In [15]:
print(df_reviews_limpio.shape)
print(df_items_limpio.shape)
print(df_games_limpio.shape)

(59333, 8)
(3285246, 4)
(32133, 10)


In [16]:
df_reviews_limpio = df_reviews_limpio.drop_duplicates()
df_items_limpio = df_items_limpio.drop_duplicates()

In [17]:
print(df_reviews_limpio.shape)
print(df_items_limpio.shape)
print(df_games_limpio.shape)

(58459, 8)
(3246362, 4)
(32133, 10)


# Renombramos columnas

In [18]:
df_items_limpio = df_items_limpio.rename(columns={'item_id': 'id'})

In [19]:
df_reviews_limpio = df_reviews_limpio.rename(columns={'item_id': 'id'})

# Creamos la columna 'sentiment_analysis'

In [20]:
from textblob import TextBlob

#columna 'review' que contiene las reseñas de los usuarios.

# Función para clasificar las reseñas
def classify_sentiment(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity

    # Clasificamos según la polaridad
    if polarity < -0.2:
        return 0  # Reseña mala
    elif polarity > 0.2:
        return 2  # Reseña positiva
    else:
        return 1  # Reseña neutra

# Aplicamos la función a la columna 'review' y guardamos los resultados en la nueva columna 'sentiment_analysis'
df_reviews_limpio['sentiment_analysis'] = df_reviews_limpio['review'].apply(classify_sentiment)


In [21]:
#Eliminamos la columna ya que fue reemplazada con la columna sentiment_analysis
df_reviews_limpio = df_reviews_limpio.drop(columns= 'review')

# FUNCION 1

In [22]:
#desanidamos la columna genres
genres_desanidado = df_games_limpio.explode('genres')

# Creamos un DF que se llama "'funcion_1" con las columnas que necesitamos para resolver la primera funcion dada

In [23]:
funcion_1i = ['id', 'playtime_forever']
funcion_1g = ['id', 'genres', 'release_date']

funcion_1idf = df_items_limpio[funcion_1i]
funcion_1gdf = genres_desanidado[funcion_1g]
funcion_1 = pd.merge(funcion_1idf, funcion_1gdf, on= 'id')

In [24]:
#Si el id y genres coinciden en varias filas el playtime_forever se suma. Asi reducimos cantidad de filas
funcion_1 = funcion_1.groupby(['id', 'genres','release_date'])['playtime_forever'].sum().reset_index()

In [25]:
#Funcion para transformar la columna de release_date en solo el año

def extraer_año(fecha):
    if isinstance(fecha, str):
        # Intentamos extraer el año si es una cadena de caracteres
        partes = fecha.split('-')
        for parte in partes:
            if parte.isdigit() and len(parte) == 4:
                return parte
    return None  # Devuelve None si no se puede extraer el año

# Aplicamos la función para extraer el año 
funcion_1['release_date'] = funcion_1['release_date'].apply(extraer_año)

In [26]:
#Funcion numero 1: devuelve el año con mas horas jugadas por GENERO
def PlayTimeGenre(genero: str):
    # Filtrar el DataFrame para obtener solo las filas del género específico
    funcion_1_filtrado = funcion_1[funcion_1['genres'].str.contains(genero, case=False, na=False)]

    if funcion_1_filtrado.empty:
        return "No se encontraron datos para el género especificado."

    # Encontrar el año con más horas jugadas
    año_con_mas_horas = funcion_1_filtrado.groupby('release_date')['playtime_forever'].sum().idxmax()

    return {"Año de lanzamiento con más horas jugadas para Género " + genero: año_con_mas_horas}


In [27]:
PlayTimeGenre('Casual')

{'Año de lanzamiento con más horas jugadas para Género Casual': '2015'}

# FUNCION 2

In [28]:
df_reviews_limpio['posted'] = df_reviews_limpio['posted'].astype(str)

In [29]:
import re

def extraer_año(fecha):
    # Utilizamos una expresión regular para buscar el año en el formato dado
    match = re.search(r'\d{4}', fecha)
    if match:
        return match.group()
    else:
        return None

# Aplicamos la función para extraer el año a la columna 'release_date'
df_reviews_limpio['posted'] = df_reviews_limpio['posted'].apply(extraer_año)

In [30]:
df_reviews_limpio['posted'] = df_reviews_limpio['posted'].fillna(0)

In [31]:
df_reviews_limpio['posted'].fillna(0, inplace=True)
df_reviews_limpio['id'].fillna(0, inplace=True)
df_items_limpio['id'].fillna(0, inplace=True)
genres_desanidado['id'].fillna(0, inplace=True)

In [32]:
df_reviews_limpio['posted'] = df_reviews_limpio['posted'].astype(int)
df_reviews_limpio['id'] = df_reviews_limpio['id'].astype(int)
df_items_limpio['id'] = df_items_limpio['id'].astype(int)
genres_desanidado['id'] = genres_desanidado['id'].astype(int)

In [33]:
#creamos el df con las columnas necesarias llamado funcion_2

funcion_2i = ['id', 'user_id', 'playtime_forever']
funcion_2r = ['user_id', 'posted']
funcion_2g = ['id', 'genres']

funcion_2idf = df_items_limpio[funcion_2i]
funcion_2rdf = df_reviews_limpio[funcion_2r]
funcion_2gdf = genres_desanidado[funcion_2g]

funcion_2 = pd.merge(funcion_2idf, funcion_2rdf, on= 'user_id')
funcion_2 = pd.merge(funcion_2, funcion_2gdf, on= 'id')

In [34]:
funcion_2 = funcion_2[funcion_2['posted'] != 0]
funcion_2 = funcion_2.drop_duplicates()

In [35]:
#Funcion numero 2: devuelve el usuario que acumula más horas jugadas para el género dado
# y una lista de la acumulación de horas jugadas por año.

def UserForGenre(genero):
    # Filtramos los datos por el género proporcionado
    data_f = funcion_2[funcion_2['genres'] == genero]

    # Calculamos las horas jugadas por usuario y año
    grouped_data = data_f.groupby(['user_id'])['playtime_forever'].sum().reset_index()

    # Encontramos al usuario con la máxima acumulación de horas jugadas
    max_user = grouped_data.groupby('user_id')['playtime_forever'].sum().idxmax()

    # Filtramos los datos para el usuario con más horas jugadas
    max_user_data = grouped_data[grouped_data['user_id'] == max_user]

    # Creamos una lista de la acumulación de horas jugadas por año
    horas_por_año = data_f.groupby('posted')['playtime_forever'].sum().reset_index()
    horas_por_año_dict = [{"Año": año, "Horas": horas} for año, horas in zip(horas_por_año['posted'], horas_por_año['playtime_forever'])]


    resultado = {
        "Usuario con más horas jugadas para el género " + genero: max_user,
        "Horas jugadas por año": horas_por_año_dict
    }

    return resultado



In [36]:
UserForGenre('Casual')

{'Usuario con más horas jugadas para el género Casual': 'REBAS_AS_F-T',
 'Horas jugadas por año': [{'Año': 2010, 'Horas': 1270625},
  {'Año': 2011, 'Horas': 4352970},
  {'Año': 2012, 'Horas': 9154282},
  {'Año': 2013, 'Horas': 26095367},
  {'Año': 2014, 'Horas': 66398704},
  {'Año': 2015, 'Horas': 63537063}]}

# FUNCION 3

In [37]:
#creamos el df con las columnas necesarias llamado funcion_3
funcion_3i = ['id', 'user_id']
funcion_3r = ['user_id', 'posted', 'recommend', 'sentiment_analysis']
funcion_3g = ['id', 'app_name']

funcion_3idf = df_items_limpio[funcion_3i]
funcion_3rdf = df_reviews_limpio[funcion_3r]
funcion_3gdf = genres_desanidado[funcion_3g]
funcion_3 = pd.merge(funcion_3idf, funcion_3rdf, on= 'user_id')
funcion_3 = pd.merge(funcion_3, funcion_3gdf, on= 'id')

In [38]:
funcion_3 = funcion_3[funcion_3['recommend'] != False]

In [39]:
funcion_3 = funcion_3[funcion_3['sentiment_analysis'] != 0]

In [40]:
funcion_3 = funcion_3.drop_duplicates()

In [41]:
def UsersRecommend(año):
    # Filtramos los datos para el año especificado
    data_filtrada = funcion_3[funcion_3['posted'] == año]

    # Calculamos la cantidad de recomendaciones para cada juego
    recomendacion = data_filtrada['app_name'].value_counts().reset_index()
    recomendacion.columns = ['app_name', 'recommend']

    # Clasificamos los juegos en función de la cantidad de recomendaciones
    top_3_games = recomendacion.sort_values(by='recommend', ascending=False).head(3)

    # Creamos la estructura de resultado deseada
    resultado = []

    for i, juego, recomendaciones in zip(range(1, 4), top_3_games['app_name'], top_3_games['recommend']):
        resultado.append(f"Puesto {i}: {juego} {recomendaciones}")

    return resultado

In [42]:
UsersRecommend(2015)

['Puesto 1: Counter-Strike: Global Offensive 8106',
 "Puesto 2: Garry's Mod 7957",
 'Puesto 3: Unturned 7211']

# FUNCION 4

In [43]:
#creamos el df con las columnas necesarias llamado funcion_4
funcion_4i = ['id', 'user_id']
funcion_4r = ['user_id', 'posted', 'recommend', 'sentiment_analysis']
funcion_4g = ['id', 'app_name']

funcion_4idf = df_items_limpio[funcion_4i]
funcion_4rdf = df_reviews_limpio[funcion_4r]
funcion_4gdf = genres_desanidado[funcion_4g]
funcion_4 = pd.merge(funcion_4idf, funcion_4rdf, on= 'user_id')
funcion_4 = pd.merge(funcion_4, funcion_4gdf, on= 'id')

In [44]:
funcion_4 = funcion_4[funcion_4['recommend'] != True]
funcion_4 = funcion_4[funcion_4['sentiment_analysis'] == 0]

In [45]:
funcion_4 = funcion_4.drop_duplicates()

In [46]:
def UsersNotRecommend(año):
    # Filtramos los datos para el año especificado
    data_filtrada = funcion_4[funcion_4['posted'] == año]

    # Calculamos la suma de recomendaciones para cada juego
    recomendaciones_por_juego = data_filtrada.groupby('app_name')['sentiment_analysis'].sum().reset_index()

    # Clasificamos los juegos en función de la suma de recomendaciones (de menor a mayor)
    juegos_menos_recomendados = recomendaciones_por_juego.sort_values(by='sentiment_analysis').head(3)

    # Creamos la estructura de resultado deseada
    resultado = [{"Puesto " + str(i + 1): juego + " " + str(recomendaciones) + " recomendaciones obtuvo"} for i, juego, recomendaciones in
                zip(range(3), juegos_menos_recomendados['app_name'], juegos_menos_recomendados['sentiment_analysis'])]

    return resultado

In [47]:
UsersNotRecommend(2012)

[{'Puesto 1': 'A Bird Story 0 recomendaciones obtuvo'},
 {'Puesto 2': 'Sins of a Solar Empire®: Rebellion 0 recomendaciones obtuvo'},
 {'Puesto 3': 'SimCity™ 4 Deluxe Edition 0 recomendaciones obtuvo'}]

# FUNCION 5

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [48]:
#creamos el df con las columnas necesarias llamado funcion_5
funcion_5i = ['id', 'user_id']
funcion_5r = ['user_id', 'sentiment_analysis']
funcion_5g = ['id', 'release_date']

funcion_5idf = df_items_limpio[funcion_5i]
funcion_5rdf = df_reviews_limpio[funcion_5r]
funcion_5gdf = genres_desanidado[funcion_5g]
funcion_5 = pd.merge(funcion_5idf, funcion_5rdf, on= 'user_id')
funcion_5 = pd.merge(funcion_5, funcion_5gdf, on= 'id')

In [49]:
funcion_5 = funcion_5.drop_duplicates()

In [50]:
funcion_5['release_date'] = funcion_5['release_date'].astype(str)
funcion_5['release_date'] = funcion_5['release_date'].apply(extraer_año)
funcion_5['release_date'].fillna(0, inplace=True)
funcion_5['release_date'] = funcion_5['release_date'].astype(int)

In [51]:
def sentiment_analysis(anio: int):
    # Filtrar los datos para el año especificado y contar las categorías de sentimiento
    funcion_5_f = funcion_5[funcion_5['release_date'] == anio]
    sentiment_counts = funcion_5_f['sentiment_analysis'].value_counts().to_dict()

    # Crear un diccionario con los resultados
    results = {
        'Negative': sentiment_counts.get(0, 0),
        'Neutral': sentiment_counts.get(1, 0),
        'Positive': sentiment_counts.get(2, 0)
    }

    return results


In [52]:
sentiment_analysis(2014)

{'Negative': 42953, 'Neutral': 165701, 'Positive': 106901}

In [53]:
# Obtener una muestra del 50% de los datos
funcion_1_muestra = funcion_1.sample(frac=0.50, random_state=1)
funcion_2_muestra = funcion_2.sample(frac=0.50, random_state=1)
funcion_3_muestra = funcion_3.sample(frac=0.50, random_state=1)
funcion_4_muestra = funcion_4.sample(frac=0.50, random_state=1)
funcion_5_muestra = funcion_5.sample(frac=0.50, random_state=1)

In [54]:
'''
funcion_1_muestra.to_parquet('funcion_1_muestra.parquet', index=True)
funcion_2_muestra.to_parquet('funcion_2_muestra.parquet', index=True)
funcion_3_muestra.to_parquet('funcion_3_muestra.parquet', index=True)
funcion_4_muestra.to_parquet('funcion_4_muestra.parquet', index=True)
funcion_5_muestra.to_parquet('funcion_5_muestra.parquet', index=True)
'''

"\nfuncion_1_muestra.to_parquet('funcion_1_muestra.parquet', index=True)\nfuncion_2_muestra.to_parquet('funcion_2_muestra.parquet', index=True)\nfuncion_3_muestra.to_parquet('funcion_3_muestra.parquet', index=True)\nfuncion_4_muestra.to_parquet('funcion_4_muestra.parquet', index=True)\nfuncion_5_muestra.to_parquet('funcion_5_muestra.parquet', index=True)\n"

In [55]:
'''
df_reviews_limpio.to_csv('df_reviews_limpio.csv', index=True)
df_items_limpio.to_csv('df_items_limpio.csv', index=True)
genres_desanidado.to_csv('genres_desanidado.csv', index=True)
'''

"\ndf_reviews_limpio.to_csv('df_reviews_limpio.csv', index=True)\ndf_items_limpio.to_csv('df_items_limpio.csv', index=True)\ngenres_desanidado.to_csv('genres_desanidado.csv', index=True)\n"